In [7]:
import time
import numpy as np
import pandas as pd
from typing import Dict, Union
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import (
    ElementClickInterceptedException,
    NoSuchElementException,
    StaleElementReferenceException,
    TimeoutException
)
from webdriver_manager.chrome import ChromeDriverManager

# from src.common.consts import CommonConsts
# from src.modules.entities.real_estate import RealEstate
# from src.modules.repositories import RealEstateRepo



# Set up Chrome driver
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
# options.add_argument("--log-level=3")  # Suppress logs
# options.add_argument("--disable-logging")  # Disable logging
# options.add_argument("--silent")  # Silent mode
# options.add_argument("headless")
options.add_argument("window-size=1920x1080")
options.add_argument("disable-gpu")
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(options=options, service=service)
driver.set_page_load_timeout(30)
link_all = [
    "https://batdongsan.com.vn/nha-dat-ban",
    "https://batdongsan.com.vn/nha-dat-cho-thue",
]
real_estate_type = [
    "Bán",
    "Cho thuê"
]
# for i in np.arange(0, len(link_all), 1):
#     real_estate_type = real_estate_type[i]
#     driver.get(link_all[i])

In [8]:
driver.get("https://batdongsan.com.vn/nha-dat-ban")

In [16]:
elems = driver.find_elements(By.CSS_SELECTOR, ".js__option .re__option")
re_types = []
for elem in elems:
    # check if ul in elem
    print(elem.text)

Chung cư mini, căn hộ dịch vụ
Nhà riêng
Nhà biệt thự, liền kề
Nhà mặt phố
Shophouse, nhà phố thương mại





In [20]:
elem = driver.find_element(By.CSS_SELECTOR, ".mCSB_container")
elem.text

'Tất cả nhà đất\nCăn hộ chung cư\nChung cư mini, căn hộ dịch vụ\nCác loại nhà bán\nNhà riêng\nNhà biệt thự, liền kề\nNhà mặt phố\nShophouse, nhà phố thương mại'

In [26]:
# Get all elem that are li tag
elems = driver.find_elements(By.CSS_SELECTOR, ".mCSB_container li")
for elem in elems:
    if elem.get_attribute("value") != "0":
        print(elem.get_attribute("value"))
        print(elem.text)


324
Căn hộ chung cư
Chung cư mini, căn hộ dịch vụ
650
Chung cư mini, căn hộ dịch vụ
362
Các loại nhà bán
Nhà riêng
Nhà biệt thự, liền kề
Nhà mặt phố
Shophouse, nhà phố thương mại
41
Nhà riêng
325
Nhà biệt thự, liền kề
163
Nhà mặt phố
575
Shophouse, nhà phố thương mại
361

40

283

44

562

45

48

